In [1]:
!nvidia-smi

Thu Oct  9 22:29:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
! pip install -q transformers[sentencepiece] fastbook fastai nbdev plum-dispatch evaluate seqeval onnxruntime onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.8 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/msi1427/blurr.git
%cd blurr

Cloning into 'blurr'...
remote: Enumerating objects: 5063, done.
remote: Counting objects: 100% (894/894), done.
remote: Compressing objects: 100% (311/311), done.
remote: Total 5063 (delta 705), reused 702 (delta 576), pack-reused 4169 (from 1)
Receiving objects: 100% (5063/5063), 26.85 MiB | 13.32 MiB/s, done.
Resolving deltas: 100% (3934/3934), done.
/content/blurr


In [4]:
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *
import pandas as pd
import torch
import json
import ast
import re
import os
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
from sklearn.metrics import multilabel_confusion_matrix, classification_report, accuracy_score, f1_score
from tqdm.notebook import tqdm
import onnxruntime as rt

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/MasterCourse/arXiv-Subjects-dataset

/content/drive/MyDrive/MasterCourse/arXiv-Subjects-dataset


In [7]:
df = pd.read_csv("arxiv_cs_papers_combined.csv")
print(f"Original shape: {df.shape}")
display(df.head(5))

Original shape: (30000, 5)


,title,abstract,subjects,url,authors
0,NFTrig,"NFTrig is a web-based application created for use as an educational tool to teach trigonometry and block chain technology. Creation of the application includes front and back end development as well as integration with other outside sources including MetaMask and OpenSea. The primary development languages include HTML, CSS (Bootstrap 5), and JavaScript as well as Solidity for smart contract creation. The application itself is hosted on Moralis utilizing their Web3 API. This technical report describes how the application was created, what the application requires, and smart contract design ...",['Human-Computer Interaction (cs.HC)'],https://arxiv.org/abs/2301.00001,"['Jordan Thompson', 'Ryan Benac', 'Kidus Olana', 'Talha Hassan', 'Andrew Sward', 'Tauheed Khan Mohd']"
1,Evaluating Alternative Glyph Design for Showing Large-Magnitude-Range Quantum Spins,"We present experimental results to explore a form of bivariate glyphs for representing large-magnitude-range vectors. The glyphs meet two conditions: (1) two visual dimensions are separable; and (2) one of the two visual dimensions uses a categorical representation (e.g., a categorical colormap). We evaluate how much these two conditions determine the bivariate glyphs' effectiveness. The first experiment asks participants to perform three local tasks requiring reading no more than two glyphs. The second experiment scales up the search space in global tasks when participants must look at th...","['Human-Computer Interaction (cs.HC)', 'Graphics (cs.GR)', 'Quantum Physics (quant-ph)']",https://arxiv.org/abs/2301.00002,"['Henan Zhao', 'Garnett W. Bryant', 'Wesley Griffin', 'Judith E. Terrill', 'Jian Chen']"
2,Emotion in Cognitive Architecture: Emergent Properties from Interactions with Human Emotion,"This document presents endeavors to represent emotion in a computational cognitive architecture. The first part introduces research organizing with two axes of emotional affect: pleasantness and arousal. Following this basic of emotional components, the document discusses an aspect of emergent properties of emotion, showing interaction studies with human users. With these past author's studies, the document concludes that the advantage of the cognitive human-agent interaction approach is in representing human internal states and processes.","['Human-Computer Interaction (cs.HC)', 'Artificial Intelligence (cs.AI)']",https://arxiv.org/abs/2301.00003,['Junya Morita']
3,Intrinsic Motivation in Dynamical Control Systems,"Biological systems often choose actions without an explicit reward signal, a phenomenon known as intrinsic motivation. The computational principles underlying this behavior remain poorly understood. In this study, we investigate an information-theoretic approach to intrinsic motivation, based on maximizing an agent's empowerment (the mutual information between its past actions and future states). We show that this approach generalizes previous attempts to formalize intrinsic motivation, and we provide a computationally efficient algorithm for computing the necessary quantities. We test our...","['Artificial Intelligence (cs.AI)', 'Applied Physics (physics.app-ph)']",https://arxiv.org/abs/2301.00005,"['Stas Tiomkin', 'Ilya Nemenman', 'Daniel Polani', 'Naftali Tishby']"
4,Recovering Top-Two Answers and Confusion Probability in Multi-Choice Crowdsourcing,"Crowdsourcing has emerged as an effective platform for labeling large amounts of data in a cost- and time-efficient manner. Most previous work has focused on designing an efficient algorithm to recover only the ground-truth labels of the data. In this paper, we consider multi-choice crowdsourcing tasks with the goal of recovering not only the ground truth, but also the most confusing answer and the confusion probability. The most confusing answer provides useful information about the task by revealing the most plausible answer other than the ground truth and how plausible it is. To

In [8]:
def clean_abstract(text):
    """Clean abstract text (remove LaTeX, URLs, punctuation, lowercase)."""
    if pd.isna(text) or not isinstance(text, str):
        return ""
    text = text.encode('utf-8', errors='ignore').decode('utf-8').lower()
    text = re.sub(r'http[s]?://\S+|this https URL', '', text)
    text = re.sub(r'\\text(sub|super)script\{([^}]*)\}', r'\2', text)
    text = re.sub(r'\$[^\$]*\$', '', text)
    text = re.sub(r'\\[a-zA-Z]+', '', text)
    text = re.sub(r'[^\w\s-]', '', text)
    return text.strip()

df['abstract'] = df['abstract'].apply(clean_abstract)
df = df[df['abstract'].str.len() > 0].reset_index(drop=True)
print(f"\n✅ Cleaned abstracts: {df.shape}")
print("Sample cleaned abstract:", df['abstract'].iloc[0][:200] + "...")


✅ Cleaned abstracts: (30000, 5)
Sample cleaned abstract: nftrig is a web-based application created for use as an educational tool to teach trigonometry and block chain technology creation of the application includes front and back end development as well as...


In [9]:

def clean_subjects(subjects_str):
    """Parse subjects into list of full names, removing invalid entries."""
    if pd.isna(subjects_str) or not isinstance(subjects_str, str):
        return []
    try:
        subjects_list = ast.literal_eval(subjects_str)
        if not isinstance(subjects_list, list):
            subjects_list = [subjects_list]
        return [s for s in subjects_list if isinstance(s, str) and s.strip()]
    except:
        return []

df['revised_subjects'] = df['subjects'].apply(clean_subjects)
df = df[df['revised_subjects'].str.len() > 0].reset_index(drop=True)
print(f"\n Cleaned subjects: {df.shape}")
print("Sample cleaned subjects:", df['revised_subjects'].iloc[0])


 Cleaned subjects: (30000, 6)
Sample cleaned subjects: ['Human-Computer Interaction (cs.HC)']


In [10]:
subjects_list = df.revised_subjects.to_list()
subject_count = {}
for subjects in subjects_list:
    subject_list = subjects
    for subject in subject_list:
        if subject in subject_count.keys():
            subject_count[subject] += 1
        else:
            subject_count[subject] = 1
print(f"Number of Subjects: {len(subject_count)}")
print("Sample subject counts:", dict(list(subject_count.items())[:]))

Number of Subjects: 141
Sample subject counts: {'Human-Computer Interaction (cs.HC)': 1389, 'Graphics (cs.GR)': 340, 'Quantum Physics (quant-ph)': 98, 'Artificial Intelligence (cs.AI)': 7613, 'Applied Physics (physics.app-ph)': 40, 'Information Theory (cs.IT)': 1209, 'Machine Learning (cs.LG)': 10053, 'Machine Learning (stat.ML)': 960, 'Neural and Evolutionary Computing (cs.NE)': 550, 'Computer Vision and Pattern Recognition (cs.CV)': 6419, 'Other Computer Science (cs.OH)': 26, 'Optimization and Control (math.OC)': 499, 'Statistics Theory (math.ST)': 104, 'Cryptography and Security (cs.CR)': 1797, 'Information Retrieval (cs.IR)': 934, 'Computers and Society (cs.CY)': 1178, 'Networking and Internet Architecture (cs.NI)': 806, 'Formal Languages and Automata Theory (cs.FL)': 139, 'Robotics (cs.RO)': 1921, 'Software Engineering (cs.SE)': 1144, 'Computation and Language (cs.CL)': 4388, 'Audio and Speech Processing (eess.AS)': 574, 'Numerical Analysis (math.NA)': 1313, 'Computational Physics

In [11]:
threshold = int(len(df) * 0.005)
rare_subjects = [key for key, value in subject_count.items() if value < threshold]
print(f"Number of rare subjects: {len(rare_subjects)}, Sample: {rare_subjects[:]}")

revised_subjects_list = []
indices_to_drop = []
for idx, subjects in enumerate(subjects_list):
    revised_subjects = [s for s in subjects if s not in rare_subjects]
    if len(revised_subjects) == 0:
        indices_to_drop.append(idx)
    else:
        revised_subjects_list.append(revised_subjects)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_subjects'] = revised_subjects_list
print(f"\n After filtering rare subjects: {df.shape}")


Number of rare subjects: 103, Sample: ['Quantum Physics (quant-ph)', 'Applied Physics (physics.app-ph)', 'Other Computer Science (cs.OH)', 'Statistics Theory (math.ST)', 'Formal Languages and Automata Theory (cs.FL)', 'Computational Physics (physics.comp-ph)', 'Operating Systems (cs.OS)', 'Symbolic Computation (cs.SC)', 'Adaptation and Self-Organizing Systems (nlin.AO)', 'Physics and Society (physics.soc-ph)', 'Mathematical Physics (math-ph)', 'Commutative Algebra (math.AC)', 'Methodology (stat.ME)', 'Computation (stat.CO)', 'Applications (stat.AP)', 'Performance (cs.PF)', 'Biomolecules (q-bio.BM)', 'Genomics (q-bio.GN)', 'Classical Analysis and ODEs (math.CA)', 'Rings and Algebras (math.RA)', 'Neurons and Cognition (q-bio.NC)', 'Dynamical Systems (math.DS)', 'Molecular Networks (q-bio.MN)', 'Fluid Dynamics (physics.flu-dyn)', 'Econometrics (econ.EM)', 'Probability (math.PR)', 'Mathematical Software (cs.MS)', 'Soft Condensed Matter (cond-mat.soft)', 'Number Theory (math.NT)', 'Computat

In [12]:
revised_subject_count = {}
for subjects in revised_subjects_list:
    for subject in subjects:
        if subject in revised_subject_count:
            revised_subject_count[subject] += 1
        else:
            revised_subject_count[subject] = 1
print(f"Number of Revised Subjects: {len(revised_subject_count)}")
print("Sample revised subject counts:", dict(list(revised_subject_count.items())[:]))

Number of Revised Subjects: 38
Sample revised subject counts: {'Human-Computer Interaction (cs.HC)': 1389, 'Graphics (cs.GR)': 340, 'Artificial Intelligence (cs.AI)': 7613, 'Information Theory (cs.IT)': 1209, 'Machine Learning (cs.LG)': 10053, 'Machine Learning (stat.ML)': 960, 'Neural and Evolutionary Computing (cs.NE)': 550, 'Computer Vision and Pattern Recognition (cs.CV)': 6419, 'Optimization and Control (math.OC)': 499, 'Cryptography and Security (cs.CR)': 1797, 'Information Retrieval (cs.IR)': 934, 'Computers and Society (cs.CY)': 1178, 'Networking and Internet Architecture (cs.NI)': 806, 'Robotics (cs.RO)': 1921, 'Software Engineering (cs.SE)': 1144, 'Computation and Language (cs.CL)': 4388, 'Audio and Speech Processing (eess.AS)': 574, 'Numerical Analysis (math.NA)': 1313, 'Computational Complexity (cs.CC)': 220, 'Data Structures and Algorithms (cs.DS)': 553, 'Systems and Control (eess.SY)': 1549, 'Social and Information Networks (cs.SI)': 629, 'Sound (cs.SD)': 557, 'Hardware A

In [13]:
encode_subject_types = {key: idx for idx, (key, value) in enumerate(revised_subject_count.items())}
with open("subject_types_encoded.json", "w") as fp:
    json.dump(encode_subject_types, fp, indent=4)
labels = list(encode_subject_types.keys())
print(f"\n Saved subject_types_encoded.json with {len(labels)} subjects")
print(f"Sample labels: {labels[:5]}")


 Saved subject_types_encoded.json with 38 subjects
Sample labels: ['Human-Computer Interaction (cs.HC)', 'Graphics (cs.GR)', 'Artificial Intelligence (cs.AI)', 'Information Theory (cs.IT)', 'Machine Learning (cs.LG)']


In [14]:
categorical_subject_list = []
for subjects in revised_subjects_list:
    categorical_list = [0] * len(labels)
    for subject in subjects:
        if subject in encode_subject_types:
            subject_idx = encode_subject_types[subject]
            categorical_list[subject_idx] = 1
    categorical_subject_list.append(categorical_list)
df['subject_cat_list'] = categorical_subject_list
print(f"Sample binary label vector: {categorical_subject_list[0][:10]}")
print(df.head())

Sample binary label vector: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                                                                                         title  \
0                                                                                       NFTrig   
1          Evaluating Alternative Glyph Design for Showing Large-Magnitude-Range Quantum Spins   
2  Emotion in Cognitive Architecture: Emergent Properties from Interactions with Human Emotion   
3                                            Intrinsic Motivation in Dynamical Control Systems   
4           Recovering Top-Two Answers and Confusion Probability in Multi-Choice Crowdsourcing   

                                                                                                                                                                                                                                                                                                                                                                

In [15]:
labels = list(encode_subject_types.keys())
len(labels), labels[:5]

(38,
 ['Human-Computer Interaction (cs.HC)',
  'Graphics (cs.GR)',
  'Artificial Intelligence (cs.AI)',
  'Information Theory (cs.IT)',
  'Machine Learning (cs.LG)'])

In [16]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(26810, 2978)

In [17]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,abstract,subjects,url,authors,revised_subjects,subject_cat_list
25966,Low-rank Prompt Interaction for Continual Vision-Language Retrieval,research on continual learning in multi-modal tasks has been receiving increasing attention however most existing work overlooks the explicit cross-modal and cross-task interactions in this paper we innovatively propose the low-rank prompt interaction lpi to address this general problem of multi-modal understanding which considers both cross-modal and cross-task interactions specifically as for the former we employ multi-modal correlation modules for corresponding transformer layers considering that the training parameters scale to the number of layers and tasks we propose low-rank interac...,['Computer Vision and Pattern Recognition (cs.CV)'],https://arxiv.org/abs/2501.14369,"['Weicai Yan', 'Ye Wang', 'Wang Lin', 'Zirun Guo', 'Zhou Zhao', 'Tao Jin']",[Computer Vision and Pattern Recognition (cs.CV)],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
29544,Reachability-Based Contingency Planning against Multi-Modal Predictions with Branch MPC,this paper presents a novel contingency planning framework that integrates learning-based multi-modal predictions of traffic participants into branch model predictive control mpc leveraging reachability analysis we address the computational challenges associated with branch mpc by organizing the multitude of predictions into driving corridors analyzing the overlap between these corridors their number can be reduced through pruning and clustering while ensuring safety since all prediction modes are preserved these processed corridors directly correspond to the distinct branches of the scena...,['Systems and Control (eess.SY)'],https://arxiv.org/abs/2502.02550,"['Mohamed-Khalil Bouzidi', 'Bojan Derajic', 'Daniel Goehring', 'Joerg Reichardt']",[Systems and Control (eess.SY)],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
13496,Learning based numerical methods for Helmholtz equation with high frequency,high-frequency issues have been remarkably challenges in numerical methods for partial differential equations in this paper a learning based numerical method lbnm is proposed for helmholtz equation with high frequency the main novelty is using tikhonov regularization method to stably learn the solution operator by utilizing relevant information especially the fundamental solutions then applying the solution operator to a new boundary input could quickly update the solution based on the method of fundamental solutions and the quantitative runge approximation we give the error estimate this ...,['Numerical Analysis (math.NA)'],https://arxiv.org/abs/2401.09118,"['Yu Chen', 'Jin Cheng', 'Tingyue Li', 'Yun Miao']",[Numerical Analysis (math.NA)],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3382,Multi-Tenant Optimization For Few-Shot Task-Oriented FAQ Retrieval,business-specific frequently asked questions faq retrieval in task-oriented dialog systems poses unique challenges vis-à-vis community based faqs each faq question represents an intent which is usually an umbrella term for many related user queries we evaluate performance for such business faqs both with standard faq retrieval techniques using query-question q-q similarity and few-shot intent detection techniques implementing a real world solution for faq retrieval in order to support multiple tenants faq sets entails optimizing speed accuracy and cost we propose a novel approach to scale ...,['Computation and Language (cs.CL)'],https://arxiv.org/abs/2301.10517,"['Asha Vishwanathan', 'Rajeev Unnikrishnan Warrier', 'Gautham Vadakkekara Suresh', 'Chandra Shekhar Kandpal']",[Computation and Language (cs.CL)],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
model_path = "models/allenai_scibert_scivocab_uncased_arxiv_multilabel_classifier.pkl"
learner_inf = load_learner(model_path)

In [19]:
learner_inf.blurr_predict("random placeholder")

[{'labels': [],
  'scores': [],
  'class_indices': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['Human-Computer Interaction (cs.HC)', 'Graphics (cs.GR)', 'Artificial Intelligence (cs.AI)', 'Information Theory (cs.IT)', 'Machine Learning (cs.LG)', 'Machine Learning (stat.ML)', 'Neural and Evolutionary Computing (cs.NE)', 'Computer Vision and Pattern Recognition (cs.CV)', 'Optimization and Control (math.OC)', 'Cryptography and Security (cs.CR)', 'Information Retrieval (cs.IR)', 'Computers and Society (cs.CY)', 'Networking and Internet Architecture (cs.NI)', 'Robotics (cs.RO)', 'Software Engineering (cs.SE)', 'Computation and Language (cs.CL)', 'Audio and Speech Processing (eess.AS)', 'Numerical Analysis (math.NA)', 'Computational Complexity (cs.CC)', 'Data Structures and Algorithms (cs.DS)'

In [20]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

[]

In [21]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['subject_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return



In [22]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['abstract']
  labels = learner_inf.blurr_predict(desc)[0]['class_indices']
  # pred_genres = [0] * len(encode_genre_types)
  # for label in labels:
  #   pred_genres[encode_genre_types[label]] = 1
  preds.append(labels)

preds[0][:20]

  0%|          | 0/2978 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.6928199791883455
F1 Score (Macro) = 0.5837293499781517


In [24]:
model_path = "models/allenai_scibert_scivocab_uncased_arxiv_multilabel_classifier.pkl"
learner_inf = load_learner(model_path)

In [25]:
learner_inf.model.hf_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [26]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/Scientific Articles-classifier.onnx',
    verbose=True,
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=14,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [27]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = 'models/Scientific Articles-classifier.onnx'
quantized_onnx_model_path = 'models/Scientific Articles-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

In [28]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

class_labels = list(encode_subject_types.keys())

inf_session = rt.InferenceSession('models/Scientific Articles-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [29]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['abstract']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_subject_types)
  for label in labels:
    pred_genres[encode_subject_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/2978 [00:00<?, ?it/s]

In [30]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.6928199791883455
F1 Score (Macro) = 0.5837293499781517


In [31]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

class_labels = list(encode_subject_types.keys())

inf_session = rt.InferenceSession('models/Scientific Articles-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [32]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.6928199791883455
F1 Score (Macro) = 0.5837293499781517
